<a href="https://colab.research.google.com/github/philippengani/shoppy/blob/main/shoppy_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's setup Spark on your Colab environment. Run the cell below!


In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!pip install findspark

     |████████████████████████████████| 212.4MB 75kB/s 
     |████████████████████████████████| 204kB 14.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=ee24b165eb9ea5cfbda20194c9a98f0cc80d998b8a4eb7c91396ea011d5d6a03
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [2]:
import pandas as pd
from pyspark import SparkConf, SparkContext, sql
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import findspark
findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [3]:
# Download the mongo db connector
!wget -P /usr/local/lib/python3.7/dist-packages/pyspark/jars https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/3.0.1/mongo-spark-connector_2.12-3.0.1.jar
!wget -P /usr/local/lib/python3.7/dist-packages/pyspark/jars https://repo1.maven.org/maven2/org/mongodb/mongo-java-driver/3.11.3/mongo-java-driver-3.11.3.jar

--2021-06-24 22:53:49--  https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/3.0.1/mongo-spark-connector_2.12-3.0.1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 552150 (539K) [application/java-archive]
Saving to: ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/mongo-spark-connector_2.12-3.0.1.jar’

mongo-spark-connect 100%[===================>] 539.21K  --.-KB/s    in 0.04s   

2021-06-24 22:53:49 (12.5 MB/s) - ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/mongo-spark-connector_2.12-3.0.1.jar’ saved [552150/552150]

--2021-06-24 22:53:49--  https://repo1.maven.org/maven2/org/mongodb/mongo-java-driver/3.11.3/mongo-java-driver-3.11.3.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209

In [4]:
# create the session and add the mongo db connector
conf = SparkConf().set("spark.ui.port", "4050").set("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")
# create the context
sc = SparkContext(conf=conf)


# Creating the database connection to the shoppy mongodb database

from pyspark.sql import SparkSession

#specifiy the directory to the jar files used for the connection to mongo db
working_directory = '/usr/local/lib/python3.7/dist-packages/pyspark/jars/*'

spark = SparkSession \
    .builder \
    .appName("mostPopularView") \
    .config('spark.driver.extraClassPath', working_directory) \
    .getOrCreate()


In [5]:
spark

In [6]:
# Creating the database connection to the shoppy mysql database

jdbcHostname = "novobyte.org"
jdbcDatabase = "jsvx9284_shpy"
username = 'jsvx9284_ndserge'
password = 'DX%A818lF.+]'
jdbcPort = 3306
jdbcUrl = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}".format(jdbcHostname, jdbcPort, jdbcDatabase, username, password)



In [7]:
# Importing the shoppy ratings data and product data

#shoppy_ratings_df = spark.read.jdbc(url=jdbcUrl, table="shoppy_ratings")
shoppy_ratings_df = spark.read.format("jdbc")\
                        .option("url", "jdbc:mysql://novobyte.org/jsvx9284_shpy?useLegacyDatetimeCode=false&serverTimezone=UTC")\
                        .option("driver", "com.mysql.jdbc.Driver")\
                        .option("dbtable", "shoppy_ratings")\
                        .option("user", "jsvx9284_ndserge")\
                        .option("password", "DX%A818lF.+]")\
                        .option("customSchema", "rating_id INT, rating_value INT, user_id INT, reference_id INT , type STRING")\
                        .load()

shoppy_products_df = spark.read.format("jdbc")\
                        .option("url", "jdbc:mysql://novobyte.org/jsvx9284_shpy?useLegacyDatetimeCode=false&serverTimezone=UTC")\
                        .option("driver", "com.mysql.jdbc.Driver")\
                        .option("query", "select product_id, product_name_en,actual_price from shoppy_products")\
                        .option("user", "jsvx9284_ndserge")\
                        .option("password", "DX%A818lF.+]")\
                        .option("customSchema", "product_id INT, product_name_en STRING, actual_price DECIMAL(38, 0)") \
                        .load()

In [8]:
# Caching both dataframes because they will be used many times
# It is better to avoid getting them all the time

shoppy_ratings_df.cache()
shoppy_products_df.cache()

print(" Shoppy ratings table : ")
shoppy_ratings_df.show(10)

print(" Shoppy products table : ")
shoppy_products_df.show(10, truncate=False)

 Shoppy ratings table : 
+---------+------------+-------+------------+-------+
|rating_id|rating_value|user_id|reference_id|   type|
+---------+------------+-------+------------+-------+
|        8|           4|     14|           4|   SHOP|
|        9|           3|     14|           1|   SHOP|
|       10|           5|     14|          35|PRODUCT|
|       11|           3|     17|          56|PRODUCT|
|       12|           3|     17|           7|   SHOP|
|       13|           2|     17|           4|   SHOP|
|       14|           3|     14|          64|PRODUCT|
|       15|           5|     41|         100|PRODUCT|
|       16|           4|     41|          64|PRODUCT|
|       17|           2|     14|          12|   SHOP|
+---------+------------+-------+------------+-------+
only showing top 10 rows

 Shoppy products table : 
+----------+-----------------------------+------------+
|product_id|product_name_en              |actual_price|
+----------+-----------------------------+------------+

In [9]:
display(shoppy_ratings_df)

DataFrame[rating_id: int, rating_value: int, user_id: int, reference_id: int, type: string]

In [10]:
shoppy_ratings_df.groupBy('reference_id').avg().head(10)

[Row(reference_id=1507, avg(rating_id)=154.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=1507.0),
 Row(reference_id=1303, avg(rating_id)=94.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=1303.0),
 Row(reference_id=1322, avg(rating_id)=123.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=1322.0),
 Row(reference_id=53, avg(rating_id)=60.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=53.0),
 Row(reference_id=1404, avg(rating_id)=143.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=1404.0),
 Row(reference_id=1378, avg(rating_id)=135.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=1378.0),
 Row(reference_id=81, avg(rating_id)=152.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=81.0),
 Row(reference_id=436, avg(rating_id)=55.0, avg(rating_value)=4.0, avg(user_id)=47.0, avg(reference_id)=436.0),
 Row(reference_id=1031, avg(rating_id)=132.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(r

In [11]:
#Get only the products
products_df = shoppy_ratings_df.where(shoppy_ratings_df.type =="PRODUCT")

In [12]:
products_df.show(10)


+---------+------------+-------+------------+-------+
|rating_id|rating_value|user_id|reference_id|   type|
+---------+------------+-------+------------+-------+
|       10|           5|     14|          35|PRODUCT|
|       11|           3|     17|          56|PRODUCT|
|       14|           3|     14|          64|PRODUCT|
|       15|           5|     41|         100|PRODUCT|
|       16|           4|     41|          64|PRODUCT|
|       18|           4|     14|         372|PRODUCT|
|       19|           0|     68|         401|PRODUCT|
|       25|           3|     26|         415|PRODUCT|
|       26|           4|     68|         440|PRODUCT|
|       27|           3|     68|         406|PRODUCT|
+---------+------------+-------+------------+-------+
only showing top 10 rows



In [13]:
# Compute the average rating value and the count for each products
from pyspark.sql import functions as F

grouped_prod_df = products_df.groupBy('reference_id').agg(F.count(products_df.rating_value).alias("count"), F.mean(products_df.rating_value).alias("average_rating"))


In [14]:
grouped_prod_df.filter("count >= 2").show(10,truncate=False)

+------------+-----+--------------+
|reference_id|count|average_rating|
+------------+-----+--------------+
|436         |2    |4.0           |
|412         |2    |3.5           |
|64          |2    |3.5           |
|437         |2    |4.0           |
|533         |2    |4.5           |
|933         |2    |1.5           |
+------------+-----+--------------+



In [15]:
# Get the grouped product df with the name of the products

grouped_prod_names = grouped_prod_df.join(shoppy_products_df,grouped_prod_df['reference_id'] == shoppy_products_df['product_id'])
grouped_prod_names.show(10)

+------------+-----+--------------+----------+--------------------+------------+
|reference_id|count|average_rating|product_id|     product_name_en|actual_price|
+------------+-----+--------------+----------+--------------------+------------+
|          35|    1|           5.0|        35| Traditional Sandals|        2000|
|          56|    1|           3.0|        56|Samsung Galaxy no...|       98000|
|          64|    2|           3.5|        64|           IPhone 5s|       80000|
|          97|    1|           4.0|        97|Customized ballerina|        3500|
|          98|    1|           4.0|        98|Flip flop, earing...|        8000|
|          99|    1|           4.0|        99| Customized hand bag|       10000|
|         100|    1|           5.0|       100|Bangles and Barrette|        2000|
|         173|    1|           3.0|       173|Polo By Ralph Lau...|        6000|
|         216|    1|           3.0|       216|      Earpeace (AKJ)|        1000|
|         260|    1|        

In [16]:
# Get select the required fields from the above table
grouped_prod_names = grouped_prod_names.select("average_rating", "product_name_en", "count", "product_id").sort('average_rating', ascending = False )

print(" The top 10 most popular products")
grouped_prod_names.show(10)




 The top 10 most popular products
+--------------+--------------------+-----+----------+
|average_rating|     product_name_en|count|product_id|
+--------------+--------------------+-----+----------+
|           5.0|deer tree jewelry...|    1|      1314|
|           5.0|Bangles and Barrette|    1|       100|
|           5.0| Traditional Sandals|    1|        35|
|           4.5|Samsung galaxy s1...|    2|       533|
|           4.0|  Ankara Flower pots|    1|       263|
|           4.0|     Alexander Shoes|    1|       348|
|           4.0|African fabric ta...|    1|       266|
|           4.0|Flip flop, earing...|    1|        98|
|           4.0|          Nike Shoes|    1|       355|
|           4.0|      Tecno camon cx|    1|       407|
+--------------+--------------------+-----+----------+
only showing top 10 rows



In [55]:
import plotly.express as px
import plotly.graph_objects as go

grouped_prod_names_pd =grouped_prod_names.toPandas()

# Get the 20 products with the largest rating

grouped_prod_names_pd = grouped_prod_names_pd.nlargest(20, 'average_rating')

'''
fig = px.bar(grouped_prod_names_pd, x='product_id', y='average_rating')
fig.show()
grouped_prod_names_pd['product_id'] = grouped_prod_names_pd['product_id'].astype(str)
'''

# Use textposition='auto' for direct text

fig = go.Figure(data=[go.Bar(
            x=grouped_prod_names_pd.product_id, y=grouped_prod_names_pd.average_rating,
            text=grouped_prod_names_pd.product_name_en,
            textposition='auto',
        )])

#Exclude all missing values
fig.update_layout(xaxis=dict(type = "category"))

fig.show()

In [18]:
print(" The list of the top 20 products :")
grouped_prod_names_pd

 The list of the top 20 products :


,average_rating,product_name_en,count,product_id
0,5.0,Traditional Sandals,1,35
1,5.0,Bangles and Barrette,1,100
2,5.0,deer tree jewelry stands,1,1314
3,4.5,Samsung galaxy s10 duos 128go,2,533
4,4.0,Customized ballerina,1,97
5,4.0,"Flip flop, earing and purse",1,98
6,4.0,Customized hand bag,1,99
7,4.0,Wall hangings,1,260
8,4.0,Wall hangings,1,262
9,4.0,Ankara Flower pots,1,263


#Most popular view

In [19]:
#Get visits data from shoppy_visits in mongo db

data = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
                    .option("spark.mongodb.input.uri", "mongodb://50.17.145.202:27017/shoppy.shoppy_visits")\
                    .load()

In [20]:
# This is a code used to connect to mongo db database using pyspark shell
'''!pyspark --conf "spark.mongodb.input.uri=mongodb://50.17.145.202:27017/shoppy.shoppy_visits?readPreference=primaryPreferred" \
         --packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1'''

'!pyspark --conf "spark.mongodb.input.uri=mongodb://50.17.145.202:27017/shoppy.shoppy_visits?readPreference=primaryPreferred"          --packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1'

In [21]:
# Get data's visitor
visitor = data.where(col('user_id').like("%log%"))

In [22]:
# Get product's visitor view 
visitor_products = visitor.where(visitor.type =="PRODUCT")

In [23]:
visitor_products.show()

+--------------------+-------------------+---------+-------+-------------------+-----------------+
|                 _id|         created_at|object_id|   type|         updated_at|          user_id|
+--------------------+-------------------+---------+-------+-------------------+-----------------+
|{60d3535297f8673a...|2021-06-23 15:29:22|      958|PRODUCT|2021-06-23 15:29:22|log_60d3535268a98|
|{60d353ec97f8673a...|2021-06-23 15:31:56|     1154|PRODUCT|2021-06-23 15:31:56|log_60d353ec37001|
|{60d353eee1324260...|2021-06-23 15:31:58|      193|PRODUCT|2021-06-23 15:31:58|log_60d353ee692e1|
|{60d353eee1324260...|2021-06-23 15:31:58|      194|PRODUCT|2021-06-23 15:31:58|log_60d353eead092|
|{60d353efe1324260...|2021-06-23 15:31:59|      195|PRODUCT|2021-06-23 15:31:59|log_60d353eeeff8b|
|{60d353efe1324260...|2021-06-23 15:31:59|      196|PRODUCT|2021-06-23 15:31:59|log_60d353ef3ecc4|
|{60d353efe1324260...|2021-06-23 15:31:59|      199|PRODUCT|2021-06-23 15:31:59|log_60d353ef81bab|
|{60d353ef

In [24]:
# Compute the count view for each products 
from pyspark.sql import functions as F

grouped_prod_visitors = visitor_products.groupBy('object_id').agg(F.count(visitor_products.object_id).alias("count"))


In [25]:
grouped_prod_visitors.where(col("count")>=5).orderBy(col("count").desc()).show()

+---------+-----+
|object_id|count|
+---------+-----+
|     1155|    7|
|      392|    7|
|     1487|    6|
|      419|    6|
|     1323|    6|
|      342|    6|
|     1324|    5|
|        6|    5|
|      299|    5|
|     1326|    5|
|      884|    5|
|      273|    5|
|      968|    5|
|      337|    5|
|      903|    5|
|      952|    5|
|      490|    5|
+---------+-----+



In [42]:
# Get the grouped product df with the name of the products

prod_names = grouped_prod_visitors.join(shoppy_products_df,grouped_prod_visitors['object_id'] == shoppy_products_df['product_id'])
prod_names.show(10)


+---------+-----+----------+--------------------+------------+
|object_id|count|product_id|     product_name_en|actual_price|
+---------+-----+----------+--------------------+------------+
|     1238|    1|      1238|Jacket set for me...|       20000|
|      471|    2|       471|Samsung Galaxy S8...|      125000|
|     1395|    2|      1395|          Blue heels|        9000|
|      392|    7|       392|       cushion cover|        3000|
|     1507|    1|      1507|               Tenis|        5000|
|      897|    2|       897|              Sandal|       20000|
|      858|    1|       858|      Coton Dentelle|       45000|
|      737|    1|       737|blue YSL  hand ba...|       17000|
|      540|    1|       540|Wax Holantex fabrics|       20000|
|     1322|    4|      1322|    Long Sleeve gown|       45000|
+---------+-----+----------+--------------------+------------+
only showing top 10 rows



[('object_id', 'int'),
 ('count', 'bigint'),
 ('product_id', 'int'),
 ('product_name_en', 'string'),
 ('actual_price', 'decimal(38,0)')]

In [43]:
# Get select the required fields from the above table
prod_names = prod_names.select("object_id", "count", "product_id", "product_name_en", "actual_price").sort('count', ascending = False )

print(" The top 10 most popular view ")
prod_names.show(10)


 The top 10 most popular view 
+---------+-----+----------+--------------------+------------+
|object_id|count|product_id|     product_name_en|actual_price|
+---------+-----+----------+--------------------+------------+
|      392|    7|       392|       cushion cover|        3000|
|     1155|    7|      1155|     LE VIEUX MOULIN|        2000|
|     1487|    6|      1487|            Lingerie|       13500|
|     1323|    6|      1323|          High heels|       37000|
|      419|    6|       419|   blue formal shoes|       25000|
|      342|    6|       342|    DSQUARED 2 shoes|       13000|
|      884|    5|       884|    Dior Bag - Black|       15000|
|        6|    5|         6|Fourgonnette 1317...|    11200000|
|     1324|    5|      1324|          High heels|       37000|
|     1326|    5|      1326|               Dress|        8000|
+---------+-----+----------+--------------------+------------+
only showing top 10 rows



[('object_id', 'int'),
 ('count', 'bigint'),
 ('product_id', 'int'),
 ('product_name_en', 'string'),
 ('actual_price', 'decimal(38,0)')]

In [54]:
prod_names_pd =prod_names.toPandas()

# Get the 20 products with the largest rating

prod_names_pd = prod_names_pd.nlargest(20, 'count')

'''
fig = px.bar(grouped_prod_names_pd, x='product_id', y='average_rating')
fig.show()
prod_names_pd['object_id'] = prod_names_pd['object_id'].astype(str)
'''

# Use textposition='auto' for direct text

fig = go.Figure(data=[go.Bar(
            x=prod_names_pd.object_id, y=prod_names_pd["count"],
            text=prod_names_pd.product_name_en,
            textposition='auto'
        )])

#Exclude all missing values
fig.update_layout(xaxis=dict(type = "category"))
fig.show()

In [41]:
prod_names_pd.dtypes

object_id          object
count               int64
product_id          int32
product_name_en    object
actual_price       object
dtype: object

object_id          object
count               int64
product_id          int32
product_name_en    object
actual_price       object
dtype: object

In [29]:
print(" The list of the top 20 of the most view  :")
prod_names_pd

 The list of the top 20 of the most view  :


,object_id,count,product_id,product_name_en,actual_price
0,392,7,392,cushion cover,3000
1,1155,7,1155,LE VIEUX MOULIN,2000
2,1323,6,1323,High heels,37000
3,419,6,419,blue formal shoes,25000
4,1487,6,1487,Lingerie,13500
5,342,6,342,DSQUARED 2 shoes,13000
6,1324,5,1324,High heels,37000
7,6,5,6,Fourgonnette 1317 fairly-used,11200000
8,299,5,299,Beautiful Brown Slippers,12000
9,1326,5,1326,Dress,8000
